In [ ]:
import os
from pprint import pprint
from importlib import import_module

from sqlalchemy.exc import IntegrityError
from sqlalchemy.orm import joinedload, subqueryload, Load, load_only
from sqlalchemy.orm.exc import NoResultFound
from sqlalchemy.dialects import postgresql

from dataservice.extensions import db
from dataservice.utils import iterate_pairwise
from dataservice import create_app
from dataservice.api.investigator.models import Investigator
from dataservice.api.study.models import Study
from dataservice.api.participant.models import Participant
from dataservice.api.biospecimen.models import Biospecimen
from dataservice.api.family.models import Family
from dataservice.api.family_relationship.models import FamilyRelationship
from dataservice.api.diagnosis.models import Diagnosis
from dataservice.api.outcome.models import Outcome
from dataservice.api.phenotype.models import Phenotype
from dataservice.api.genomic_file.models import GenomicFile
from dataservice.api.sequencing_experiment.models import SequencingExperiment
from dataservice.api.workflow.models import Workflow, WorkflowGenomicFile
from dataservice.api.study_file.models import StudyFile

from dataservice.util.data_import.utils import to_camel_case
from dataservice.util.data_import.etl.defaults import DEFAULT_ENTITY_TYPES

study_id = 'SD_FN5YSGZE'

class BaseLoader(object):

    def __init__(self, config_name=None):
        if not config_name:
            config_name = 'testing'
        self.setup(config_name)
        self.entity_id_map = {}

    def setup(self, config_name):
        """
        Creates tables in database
        """
        self.app = create_app(config_name)
        self.app.config['SQLALCHEMY_ECHO'] = True
        self.app_context = self.app.app_context()
        self.app_context.push()
        db.drop_all()
        db.create_all()
#         self.import_models()

    def teardown(self):
        """
        Clean up
        """
        db.session.close()
        db.drop_all()

    def drop_all(self, study_external_id):
        """
        Delete all data related to a study
        """
        from dataservice.api.study.models import Study
        from dataservice.api.investigator.models import Investigator

        try:
            study = Study.query.filter_by(external_id=study_external_id).one()
        except NoResultFound:
            print("Study {} not found. Aborting drop all for this dataset"
                  .format(study_external_id))
        else:
            # Save investigator id
            investigator_id = study.investigator_id

            # Delete study
            db.session.delete(study)

            # Delete investigator
            if investigator_id:
                investigator = Investigator.query.get(investigator_id)
                db.session.delete(investigator)

            db.session.commit()

    def import_models(self, skip_entities=[]):
        """
        Load all entities into db
        """
        # For each entity type
        for entity_type in DEFAULT_ENTITY_TYPES:
            # Skip some entities
            if entity_type in skip_entities:
                continue
            # Dynamically import entity model class
            model_name = to_camel_case(entity_type)
            model_module_path = 'dataservice.api.{}.models'.format(
                entity_type)
            models_module = import_module(model_module_path)
            model = getattr(models_module, model_name)

In [ ]:
loader = BaseLoader()

In [ ]:
# Add a bunch of studies for pagination
for i in range(101):
    s = Study(external_id='Study_{}'.format(i))
    db.session.add(s)
db.session.commit()

# Add a bunch of study files
s0 = Study.query.filter_by(external_id='Study_0').one()
s1 = Study.query.filter_by(external_id='Study_1').one()
for i in range(101):
    sf = StudyFile(file_name='blah', study_id=s0.kf_id)
    db.session.add(sf)
db.session.commit()

# Add a bunch of investigators
for _ in range(102):
    inv = Investigator(name='test')
    inv.studies.extend([s0, s1])
    db.session.add(inv)
db.session.commit()

# Add a bunch of families
families = []
for i in range(101):
    families.append(Family(external_id='Family_{}'.format(i)))
db.session.add_all(families)
db.session.commit()

participants = []
f0 = Family.query.filter_by(external_id='Family_0').one()
f1 = Family.query.filter_by(external_id='Family_1').one()
for i in range(102):
    f = f0 if i < 50 else f1
    s = s0 if i < 50 else s1
    data = {
        'external_id': "test",
        'is_proband': True,
        'consent_type': 'GRU-IRB',
        'race': 'asian',
        'ethnicity': 'not hispanic',
        'gender': 'male'
    }

    p = Participant(**data, study_id=s.kf_id, family_id=f.kf_id)
    samp = Biospecimen(analyte_type='an analyte')
    se_kwargs = {
        'external_id': 'se1',
        'experiment_strategy': 'WGS',
        'center': 'Baylor',
        'is_paired_end': True,
        'platform': 'Illumina'
    }
    seq_exp = SequencingExperiment(**se_kwargs)
    gf = GenomicFile()
    gf.biospecimen = samp
    gf.sequencing_experiment = seq_exp
    p.biospecimens = [samp]
    diag = Diagnosis()
    p.diagnoses = [diag]
    outcome = Outcome()
    p.outcomes = [outcome]
    phen = Phenotype()
    p.phenotypes = [phen]
    participants.append(p)
    db.session.add(p)
db.session.commit()

# Family relationships
for participant, relative in iterate_pairwise(participants):
    gender = participant.gender
    rel = 'mother'
    if gender == 'male':
        rel = 'father'
    r = FamilyRelationship(participant=participant, relative=relative,
                           participant_to_relative_relation=rel)
    db.session.add(r)
db.session.commit()

In [ ]:
study_id = Study.query.filter_by(external_id='Study_0').one().kf_id

### Wrong way to load children through joins

In [ ]:
q = (Diagnosis.query.options(joinedload(Diagnosis.participant, innerjoin=True).load_only('kf_id'))
# .options(Load(Participant).load_only('kf_id', 'study_id'))
.filter(Participant.study_id == study_id))
print(q.statement.compile(dialect=postgresql.dialect()))

In [ ]:
q = (Diagnosis.query
     .join(Participant.diagnoses)
     .options(Load(Participant).load_only('kf_id', 'study_id'))
     .filter(Participant.study_id == study_id))
print(q.statement.compile(dialect=postgresql.dialect()))

In [ ]:
# q = (Diagnosis.query
#      .options(joinedload(Participant.diagnoses).load_only('kf_id')))
#      .options(Load(Participant).load_only('kf_id', 'study_id'))
#      .filter(Participant.study_id == study_id))
# print(q.statement.compile(dialect=postgresql.dialect()))

In [ ]:
q = (GenomicFile.query.options(
     joinedload(GenomicFile.sequencing_experiment).load_only("kf_id")
     .joinedload(SequencingExperiment.aliquot).load_only("kf_id")
     .joinedload(Aliquot.sample).load_only("kf_id"))
     .join(Sample.participant).options(Load(Participant).load_only("kf_id", "study_id"))
     .filter(Participant.study_id==study_id))
print(q.statement.compile(dialect=postgresql.dialect()))

### Correct way to load children through joins

In [ ]:
# Genomic files
q = (GenomicFile.query
     .join(SequencingExperiment.genomic_files)
     .join(Aliquot.sequencing_experiments)
     .join(Sample.aliquots)
     .join(Participant.samples)
     .filter(Participant.study_id==study_id)
    )
print(q.statement.compile(dialect=postgresql.dialect()))

In [ ]:
# Participants
q = (Participant.query
                .options(joinedload(Participant.diagnoses)
                        .load_only('kf_id'))
                .options(joinedload(Participant.samples)
                        .load_only('kf_id'))
                .options(joinedload(Participant.phenotypes)
                        .load_only('kf_id'))
                .options(joinedload(Participant.outcomes)
                        .load_only('kf_id')))
print(q.statement.compile(dialect=postgresql.dialect()))

In [ ]:
# Family 
q = (Family.query
     .join(Family.participants)
    .options(Load(Participant).load_only('kf_id', 'study_id'))
    .filter(Participant.study_id==study_id)
    .distinct(Family.kf_id)
    .order_by(Family.kf_id))

In [ ]:
print(q.statement.compile(dialect=postgresql.dialect()))
q.count()

In [ ]:
# Family relationship
q = (FamilyRelationship.query
     .join(FamilyRelationship.participant)
    .options(Load(Participant).load_only('kf_id', 'study_id'))
    .filter(Participant.study_id==study_id))
print(q.statement.compile(dialect=postgresql.dialect()))
q.count()

In [ ]:
# Study File
q = StudyFile.query.filter(StudyFile.study_id == study_id)
print(q.statement.compile(dialect=postgresql.dialect()))
results = q.all()

In [ ]:
# Investigator
q = (Investigator.query
     .join(Investigator.studies)
     .options(Load(Study).load_only('kf_id'))
     .filter(Study.kf_id==study_id))
print(q.statement.compile(dialect=postgresql.dialect()))
q.count()

In [ ]:
# Genomic files
q = (GenomicFile.query
     .join(GenomicFile.biospecimen)
     .join(Biospecimen.participant)
     .options(Load(Participant).load_only("kf_id", "study_id"))
     .filter(Participant.study_id==study_id))
print(q.statement.compile(dialect=postgresql.dialect()))
q.count()

In [ ]:
# Biospecimen
q = (Biospecimen.query
     .join(Participant.biospecimens)
      .options(Load(Participant).load_only('kf_id', 'study_id'))
      .filter(Participant.study_id == study_id))
print(q.statement.compile(dialect=postgresql.dialect()))
q.count()

In [ ]:
# Sequencing experiment
q = (SequencingExperiment.query
     .join(SequencingExperiment.genomic_files)
     .join(GenomicFile.biospecimen)
     .join(Biospecimen.participant)
     .options(Load(Participant).load_only("kf_id", "study_id"))
     .filter(Participant.study_id==study_id))
print(q.statement.compile(dialect=postgresql.dialect()))
q.count()

In [ ]:
db.drop_all()